In [1]:
import pandas as pd
import numpy as np
from scipy.linalg import sqrtm
from sklearn.metrics import accuracy_score
import time
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
import warnings
warnings.filterwarnings("ignore")
from scipy.spatial.distance import pdist,squareform,euclidean
from scipy.spatial import distance_matrix
from scipy.sparse.linalg import eigsh
from scipy.stats import unitary_group
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.manifold.t_sne import (_joint_probabilities,
                                    _kl_divergence)


np.random.seed(123)

In [2]:
def _joint_probabilities_constant_sigma(D, sigma):
    P = np.exp(-D**2/2 * sigma**2)
    P /= np.sum(P, axis=1)
    return P

In [3]:
def similarity_func(u, v):
    return 1/(1+euclidean(u,v))

My version of MSC, implemented via the algorithm description in the paper. It is rather slow, so that we have decided to replace it by the original code by Stella Yu (2003), below.

In [17]:
def MSC(X, n_classes, n_steps=1000, tol=1e-5):
    '''
    Implementation of Direct Normalized Cut (Algorithm 1) from the article.
    
    Attributes:
    ---
    X: ndarray
        Matrix (dxn), d is the number of features, n is the number of observations.
    n_classes: int
        Number of clusters.
    n_steps: int
        Number of iterations.
    tol: float
        Precision of the algorithm.
    
    Returns:
    ---
    Y: ndarray
         Matrix (nxc) with predicted clusters.
    '''
    D = pairwise_distances(X, squared=True)
    A = _joint_probabilities_constant_sigma(D, .002)
    #A = squareform(pdist(X, similarity_func))       
                              
    D_A = np.diag(A.sum(axis=1))
    D_A_inv_0_5 = np.diag(np.power(A.sum(axis=1), -0.5))
    D_A_0_5 = np.diag(np.power(A.sum(axis=1), 0.5))
    
    Y = np.random.rand(A.shape[0], n_classes)
    row_maxes = Y.max(axis=1).reshape(-1, 1)
    Y[:] = np.where(Y == row_maxes, 1, 0)
    
    for i in range(n_steps):
        Z = Y @ np.linalg.pinv(sqrtm(Y.T @ D_A @ Y))
    
        eigen_val, eigen_vec = eigsh(np.linalg.pinv(D_A) @ A, n_classes, maxiter = 1000, tol = tol, which = 'LA')
    
        i = np.argsort(-eigen_val)
        eigen_val = eigen_val[i]
        eigen_vec = eigen_vec[:,i]
    
        eigen_vec = D_A * np.matrix(eigen_vec)
        norm_ones = np.linalg.norm(np.ones((A.shape[0], 1)))
        for i in range(0, eigen_vec.shape[1]):
            eigen_vec[:,i] = (eigen_vec[:,i] / np.linalg.norm(eigen_vec[:,i])) * norm_ones
        if eigen_vec[0,i] != 0:
                eigen_vec[:,i] = -1 * eigen_vec[:,i] * np.sign( eigen_vec[0,i] )
    
        c = np.zeros((A.shape[0], 1))
        R = np.matrix(np.zeros((n_classes, n_classes)))
        R[:, 0] = eigen_vec[int(np.random.rand(1) * (A.shape[0] - 1)),:].transpose()

        for j in range(1,n_classes):
            c = c + abs(eigen_vec * R[:, j - 1])
            R[:, j]=eigen_vec[c.argmin(), :].transpose()
        Z_star = np.matrix(eigen_vec) @ R
    
        Y_star = sqrtm(np.linalg.pinv(np.diag(np.diag(Z_star @ Z_star.T)))) @ Z_star
        Y_0 = Y
        Y = Y_star @ R
        
        if  np.linalg.norm(Y_0 - Y, "fro") < tol:
            break
        
    return Y

In [5]:
def accuracy(clusters_labels, y_true):
    '''
    Function for measuring accuracy of clusterization.
    
    Attributes:
    ---
    clusters_labels: ndarray
        Predicted labels, size: (1xc).
    y_true: ndarray
        True labels, size: (1xc).
    
    Returns:
    ---
    accur: float
        Accuracy of clusterization.
    '''
    clusters = np.unique(clusters_labels)
    y_pred = np.empty(len(y_true))
    
    for cluster in clusters:
        mask_cluster = clusters_labels == cluster
        mean_pred = np.bincount(y_true[mask_cluster]).argmax()
        y_pred[mask_cluster] = mean_pred
    accur = accuracy_score(y_true, y_pred)
    return accur

The Stella Yu (2003) MSC implementation: it is faster, so we will perform the experiments with it!

In [480]:
#### python_ncut_lib.py
# Copyright (C) 2010 R. Cameron Craddock (cameron.craddock@gmail.com)
#
# This script is a part of the pyClusterROI python toolbox for the spatially
# constrained clustering of fMRI data. It provides the library functions for
# performing normalized cut clustering according to:
#
# Stella Yu and Jianbo Shi, "Understanding Popout through Repulsion," Computer
# Vision and Pattern Recognition, December, 2001.  
#
# Shi, J., & Malik, J. (2000).  Normalized cuts and image segmentation. IEEE
# Transactions on Pattern Analysis and Machine Intelligence, 22(8), 888-905.
# doi: 10.1109/34.868688.
#
# Yu, S. X., & Shi, J. (2003). Multiclass spectral clustering. Proceedings Ninth
# IEEE International Conference on Computer Vision, (1), 313-319 vol.1. Ieee.
# doi: 10.1109/ICCV.2003.1238361.
#
# This code is a port of the NcutClustering_7 matlab toolbox available here:
# http://www.cis.upenn.edu/~jshi/software/
#
# For more information refer to:
#
# Craddock, R. C.; James, G. A.; Holtzheimer, P. E.; Hu, X. P. & Mayberg, H. S.
# A whole brain fMRI atlas generated via spatially constrained spectral
# clustering Human Brain Mapping, 2012, 33, 1914-1928 doi: 10.1002/hbm.21333.
#
# ARTICLE{Craddock2012,
#   author = {Craddock, R C and James, G A and Holtzheimer, P E and Hu, X P and
#   Mayberg, H S},
#   title = {{A whole brain fMRI atlas generated via spatially constrained
#   spectral clustering}},
#   journal = {Human Brain Mapping},
#   year = {2012},
#   volume = {33},
#   pages = {1914--1928},
#   number = {8},
#   address = {Department of Neuroscience, Baylor College of Medicine, Houston,
#       TX, United States},
#   pmid = {21769991},
# } 
#
# Documentation, updated source code and other information can be found at the
# NITRC web page: http://www.nitrc.org/projects/cluster_roi/ and on github at
# https://github.com/ccraddock/cluster_roi
#
# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
# 
# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with this program.  If not, see <http://www.gnu.org/licenses/>.
####

# this scripts requires NumPy (numpy.scipy.org) and SciPy (www.scipy.org) to be
# installed in a directory that is accessible through PythonPath 
import sys
from numpy import array, reshape, shape, matrix, ones, zeros, sqrt
from numpy import argsort, sign, kron, multiply, divide, abs, asarray
from scipy import rand
from scipy.sparse import csc_matrix, spdiags
from scipy.sparse.linalg import eigsh
from scipy.linalg import norm, svd, LinAlgError

# exception hander for singular value decomposition
class SVDError(Exception):
    def __init__(self,value):
        self.value=value
    def __str__(self):
        return repr(self.value)


# (eigen_val, eigen_vec) = ncut( W, nbEigenValues ):
#
# This function performs the first step of normalized cut spectral clustering.
# The normalized LaPlacian is calculated on the similarity matrix W, and top
# nbEigenValues eigenvectors are calculated. The number of eigenvectors
# corresponds to the maximum number of classes (K) that will be produced by the
# clustering algorithm. 
#
#    W:             symmetric #feature x #feature sparse matrix representing the
#                   similarity between voxels, traditionally this matrix should
#                   be positive semidefinite, but regularization is employed to
#                   allow negative matrix entries (Yu 2001)
#    nvEigenValues: number of eigenvectors that should be calculated, this
#                   determines the maximum number of clusters (K) that can be
#                   derived from the
#    result
#    eigen_val:     (output) eigenvalues from the eigen decomposition of the
#                   LaPlacian of W
#    eigen_vec:     (output) eigenvectors from the eign decomposition of the
#                   LaPlacian of W
#
def ncut( W, nbEigenValues ):
    # parameters
    offset=.5
    maxiterations=100
    eigsErrorTolerence=1e-6
    eps=2.2204e-16

    m=shape(W)[1]

    # make sure that W is symmetric, this is a computationally expensive
    # operation, only use for debugging
    # if (W-W.transpose()).sum() != 0:
    #    print "W should be symmetric!"
    #    exit(0)

    # degrees and regularization
    # S Yu Understanding Popout through Repulsion CVPR 2001
    # Allows negative values as well as improves invertability of d for small
    # numbers i bet that this is what improves the stability of the eigen
    d=abs(W).sum(0)
    dr=0.5*(d-W.sum(0))
    d=d+offset*2
    dr=dr+offset

    # calculation of the normalized LaPlacian
    W=W+spdiags(dr,[0],m,m,"csc")
    Dinvsqrt=spdiags((1.0/sqrt(d+eps)),[0],m,m,"csc")
    P=Dinvsqrt*(W*Dinvsqrt);

    # perform the eigen decomposition
    eigen_val,eigen_vec=eigsh(P,nbEigenValues,maxiter=maxiterations,\
        tol=eigsErrorTolerence,which='LA')

    # sort the eigen_vals so that the first
    # is the largest
    i=argsort(-eigen_val)
    eigen_val=eigen_val[i]
    eigen_vec=eigen_vec[:,i]

    # normalize the returned eigenvectors
    eigen_vec=Dinvsqrt*matrix(eigen_vec)
    norm_ones=norm(ones((m,1)))
    for i in range(0,shape(eigen_vec)[1]):
        eigen_vec[:,i]=(eigen_vec[:,i] / norm(eigen_vec[:,i]))*norm_ones
        if eigen_vec[0,i] != 0:
            eigen_vec[:,i] = -1 * eigen_vec[:,i] * sign( eigen_vec[0,i] )

    return(eigen_val, eigen_vec)

# eigenvec_discrete=discretisation( eigen_vec ):
#
# This function performs the second step of normalized cut clustering which
# assigns features to clusters based on the eigen vectors from the LaPlacian of
# a similarity matrix. There are a few different ways to perform this task. Shi
# and Malik (2000) iteratively bisect the features based on the positive and
# negative loadings of the eigenvectors. Ng, Jordan and Weiss (2001) proposed to
# perform K-means clustering on the rows of the eigenvectors. The method
# implemented here was proposed by Yu and Shi (2003) and it finds a discrete
# solution by iteratively rotating a binaised set of vectors until they are
# maximally similar to the the eigenvectors (for more information, the full
# citation is at the top of this file). An advantage of this method over K-means
# is that it is _more_ deterministic, i.e. you should get very similar results
# every time you run the algorithm on the same data.
#
# The number of clusters that the features are clustered into is determined by
# the number of eignevectors (number of columns) in the input array eigen_vec. A
# caveat of this method, is that number of resulting clusters is bound by the
# number of eignevectors, but it may contain less.
#
#    eigen_vec:          Eigenvectors of the normalized LaPlacian calculated
#                        from the similarity matrix for the corresponding
#                        clustering problem
#    eigen_vec_discrete: (output) discretised eigenvectors, i.e. vectors of 0
#                        and 1 which indicate whether or not a feature belongs
#                        to the cluster defined by the eigen vector.  I.E. a one
#                        in the 10th row of the 4th eigenvector (column) means
#                        that feature 10 belongs to cluster #4.
# 
def discretisation( eigen_vec ):
    eps=2.2204e-16

    # normalize the eigenvectors
    [n,k]=shape(eigen_vec)
    vm=kron(ones((1,k)),sqrt(multiply(eigen_vec,eigen_vec).sum(1)))
    eigen_vec=divide(eigen_vec,vm)

    svd_restarts=0
    exitLoop=0

    ### if there is an exception we try to randomize and rerun SVD again
        ### do this 30 times
    while (svd_restarts < 30) and (exitLoop==0):

        # initialize algorithm with a random ordering of eigenvectors
        c=zeros((n,1))
        R=matrix(zeros((k,k)))
        R[:,0]=eigen_vec[int(rand(1)*(n-1)),:].transpose()

        for j in range(1,k):
            c=c+abs(eigen_vec*R[:,j-1])
            R[:,j]=eigen_vec[c.argmin(),:].transpose()


        lastObjectiveValue=0
        nbIterationsDiscretisation=0
        nbIterationsDiscretisationMax=20

        # iteratively rotate the discretised eigenvectors until they
        # are maximally similar to the input eignevectors, this 
        # converges when the differences between the current solution
        # and the previous solution differs by less than eps or we
        # we have reached the maximum number of itarations
        while exitLoop == 0:
            nbIterationsDiscretisation = nbIterationsDiscretisation + 1

            # rotate the original eigen_vectors
            tDiscrete=eigen_vec*R

            # discretise the result by setting the max of each row=1 and
            # other values to 0
            j=reshape(asarray(tDiscrete.argmax(1)),n)
            eigenvec_discrete=csc_matrix((ones(len(j)),(range(0,n), \
                array(j))),shape=(n,k))

            # calculate a rotation to bring the discrete eigenvectors cluster to
            # the original eigenvectors
            tSVD=eigenvec_discrete.transpose()*eigen_vec
            # catch a SVD convergence error and restart
            try:
                U, S, Vh = svd(tSVD)
            except LinAlgError:
                # catch exception and go back to the beginning of the loop
                print >> sys.stderr, \
                    "SVD did not converge, randomizing and trying again"
                break

            # test for convergence
            NcutValue=2*(n-S.sum())
            if((abs(NcutValue-lastObjectiveValue) < eps ) or \
                      ( nbIterationsDiscretisation > \
                        nbIterationsDiscretisationMax )):
                exitLoop=1
            else:
                # otherwise calculate rotation and continue
                lastObjectiveValue=NcutValue
                R=matrix(Vh).transpose()*matrix(U).transpose()

    if exitLoop == 0:
        raise SVDError("SVD did not converge after 30 retries")
    else:
        return(eigenvec_discrete)

In [512]:
X, y = make_blobs()
pred = ncut(squareform(pdist(X, similarity_func)), 3)
predictions = np.array([])
for i in range(pred[1].shape[0]):
    predictions = np.append(predictions, np.argmax(pred[1][i,:]))
accuracy_score(y, predictions)

0.34

In [481]:
def similarity_func(u, v):
    return np.exp(- euclidean(u,v) ** 2 / 2)

In [492]:
data = pd.read_csv("C:/Users/Yuliya/Downloads/isolet5.data", header=None)
X = data.iloc[:, :-1]
y = data.iloc[:, -1]
start = time.time()
pred = ncut(squareform(pdist(X, similarity_func)), 26)
print(time.time() - start)

predictions = np.array([])
for i in range(pred[1].shape[0]):
    predictions = np.append(predictions, np.argmax(pred[1][i,:]))
accuracy(predictions, y)

166.582288980484


0.11096856959589481

In [493]:
start = time.time()
pred = ncut(squareform(pdist(X, similarity_func)), 13)
print(time.time() - start)

predictions = np.array([])
for i in range(pred[1].shape[0]):
    predictions = np.append(predictions, np.argmax(pred[1][i,:]))
accuracy(predictions, y)

188.80339741706848


0.06991661321359846

In [494]:
start = time.time()
pred = ncut(squareform(pdist(X, similarity_func)), 30)
print(time.time() - start)

predictions = np.array([])
for i in range(pred[1].shape[0]):
    predictions = np.append(predictions, np.argmax(pred[1][i,:]))
accuracy(predictions, y)

190.41396379470825


0.09364977549711354

In [495]:
segments = pd.read_csv("C:/Users/Yuliya/Downloads/segments.csv")
y = np.array(segments["class"])
X = segments.drop(["class"], axis=1)
X = np.array(X)
start = time.time()
pred = ncut(squareform(pdist(X, similarity_func)), 7)
print(time.time() - start)

predictions = np.array([])
for i in range(pred[1].shape[0]):
    predictions = np.append(predictions, np.argmax(pred[1][i,:]))
accuracy(predictions, y)

349.56039929389954


0.16536796536796536

In [496]:
start = time.time()
pred = ncut(squareform(pdist(X, similarity_func)), 3)
print(time.time() - start)

predictions = np.array([])
for i in range(pred[1].shape[0]):
    predictions = np.append(predictions, np.argmax(pred[1][i,:]))
accuracy(predictions, y)

351.7164535522461


0.15887445887445886

In [497]:
start = time.time()
pred = ncut(squareform(pdist(X, similarity_func)), 9)
print(time.time() - start)

predictions = np.array([])
for i in range(pred[1].shape[0]):
    predictions = np.append(predictions, np.argmax(pred[1][i,:]))
accuracy(predictions, y)

381.71722507476807


0.1619047619047619

In [487]:
glass = pd.read_csv("C:/Users/Yuliya/Downloads/glass_processed.csv")
y = glass["target"]
X = glass.iloc[:, :-1]

In [489]:
start = time.time()
pred = ncut(squareform(pdist(X, similarity_func)), 6)
print(time.time() - start)

predictions = np.array([])
for i in range(pred[1].shape[0]):
    predictions = np.append(predictions, np.argmax(pred[1][i,:]))
accuracy(predictions, y)

2.824214220046997


0.3598130841121495

In [490]:
start = time.time()
pred = ncut(squareform(pdist(X, similarity_func)), 9)
print(time.time() - start)

predictions = np.array([])
for i in range(pred[1].shape[0]):
    predictions = np.append(predictions, np.argmax(pred[1][i,:]))
accuracy(predictions, y)

3.0472254753112793


0.4392523364485981

In [491]:
start = time.time()
pred = ncut(squareform(pdist(X, similarity_func)), 3)
print(time.time() - start)

predictions = np.array([])
for i in range(pred[1].shape[0]):
    predictions = np.append(predictions, np.argmax(pred[1][i,:]))
accuracy(predictions, y)

3.0262415409088135


0.37850467289719625

In [498]:
mnist=pd.read_csv('https://pkgstore.datahub.io/machine-learning/mnist_784/mnist_784_csv/data/89c44af4c515d5a3c132bc3cc298a6bd/mnist_784_csv.csv')
X_mnist=mnist.drop('class',axis=1)
y_mnist=mnist['class']
X_mnist=X_mnist[:6000]
y_mnist=y_mnist[:6000]

In [499]:
start = time.time()
pred = ncut(squareform(pdist(X_mnist, similarity_func)), 10)
print(time.time() - start)

predictions = np.array([])
for i in range(pred[1].shape[0]):
    predictions = np.append(predictions, np.argmax(pred[1][i,:]))

3668.404913663864


IndexError: boolean index did not match indexed array along dimension 0; dimension is 2310 but corresponding boolean dimension is 6000

I have forgot to change inputs of the "accuracy" function, therefore it is run below:

In [507]:
accuracy(predictions, y_mnist)

0.122